In [1]:
import gzip
import numpy as np
import struct

In [2]:
with gzip.GzipFile('DAQ_2025-06-04T18-42-06/DATA_0/EOS.gz', 'rb') as f:
    buf = f.read(1_000_000_000)

In [3]:
it = struct.iter_unpack(">LQ2s", buf)

In [4]:
ev = list(it)

In [5]:
np.frombuffer(ev[0][-1], dtype=np.float16), np.frombuffer(b'\x00D', dtype=np.float16)

(array([nan], dtype=float16), array([4.], dtype=float16))

In [6]:
ev

[(1, 1749026529879394740, b'\x00~'),
 (1, 1749026529884295336, b'\x00D'),
 (2, 1749026529921066369, b'\x00~'),
 (2, 1749026529926019447, b'\x00D'),
 (3, 1749026529962868961, b'\x00~'),
 (3, 1749026529967840410, b'\x00D'),
 (4, 1749026530004997906, b'\x00~'),
 (4, 1749026530009403077, b'\x00D'),
 (5, 1749026530046350165, b'\x00~'),
 (5, 1749026530051289873, b'\x00D'),
 (6, 1749026530088457368, b'\x00~'),
 (6, 1749026530092930243, b'\x00D'),
 (7, 1749026530129957831, b'\x00~'),
 (7, 1749026530134396057, b'\x00D'),
 (8, 1749026530171446442, b'\x00~'),
 (8, 1749026530176117538, b'\x00D'),
 (9, 1749026530212675182, b'\x00~'),
 (9, 1749026530217581520, b'\x00D'),
 (10, 1749026530254401311, b'\x00~'),
 (10, 1749026530259292204, b'\x00D'),
 (11, 1749026530296536034, b'\x00~'),
 (11, 1749026530301421223, b'\x00D'),
 (12, 1749026530338267793, b'\x00~'),
 (12, 1749026530342662204, b'\x00D'),
 (13, 1749026530379868459, b'\x00~'),
 (13, 1749026530384288463, b'\x00D'),
 (14, 1749026530421297773, b'\

In [7]:
t0 = [ev[i][1] for i in range(len(ev)) if ev[i][-1] == b'\x00~']
t4 = [ev[i][1] for i in range(len(ev)) if ev[i][-1] == b'\x00D']

In [8]:
t0_s = np.array(t0) / 1e9
t4_s = np.array(t4) / 1e9

In [9]:
np.mean(1/np.diff(t0_s)), np.mean(1/np.diff(t4_s))

(np.float64(24.00051367591576), np.float64(24.00021171218282))

In [10]:
# t0_s -= start
# t4_s -= start

t4_s -= t0_s[0]
t0_s -= t0_s[0]


In [12]:
(42.18/360) * (1/24) * 1000

4.881944444444444

In [13]:
t4_s - t0_s

array([0.00490069, 0.00495291, 0.0049715 , 0.00440526, 0.00493979,
       0.00447273, 0.00443816, 0.00467086, 0.00490642, 0.00489092,
       0.00488544, 0.00439429, 0.00442004, 0.00505877, 0.00471783,
       0.0048883 , 0.00437427, 0.00488925, 0.0048759 , 0.00460029,
       0.00480342, 0.00479841, 0.00476241, 0.00528145, 0.00484419,
       0.00472689, 0.00489736, 0.0046978 , 0.00475597, 0.00483179,
       0.00498104, 0.00489497, 0.00475216, 0.00442934, 0.00496078,
       0.0045259 , 0.00439358, 0.00492239, 0.00489926, 0.0049057 ,
       0.00498152, 0.004915  , 0.00534201, 0.00501752, 0.00489664,
       0.00469923, 0.00488424, 0.00490761, 0.00493717, 0.00537515,
       0.0049448 , 0.00491309, 0.00491333, 0.00456238, 0.00501871,
       0.00478101, 0.00491619, 0.00537729, 0.00489879, 0.00445843,
       0.00493312, 0.00492287, 0.00501609, 0.00490141, 0.00490594,
       0.00490952, 0.00491405, 0.00491691, 0.00491047, 0.00506282,
       0.00443363, 0.00497556, 0.00490427, 0.00491023, 0.00490

In [14]:
(t4_s * 1000)[:10]

array([  4.90069389,  46.62466049,  88.44566345, 130.00845909,
       171.89502716, 213.53554726, 255.00130653, 296.72265053,
       338.18674088, 379.89735603])

In [15]:
(t0_s * 1000)[:10]

array([  0.        ,  41.67175293,  83.47415924, 125.60319901,
       166.95523262, 209.06281471, 250.56314468, 292.05179214,
       333.28032494, 375.0064373 ])

In [16]:
np.mean((t4_s * 1000) - (t0_s * 1000))

np.float64(4.887687975281077)